In [ ]:
%load_ext autoreload
%autoreload 2
import nima_io
import nima_io.read as ir

# import skimage.io
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# nd2 = "/home/dan/pims/pims/tests/data/bioformats/cluster.nd2"
import numpy as np
from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
# imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

# mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
# bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
# simgsingle = str(imgsingle)
# smcts = str(mcts)
# sbigtiff = str(bigtiff)

Core metadata list at:
https://docs.openmicroscopy.org/bio-formats/5.9.1/developers/file-reader.html



In [ ]:
ir.ensure_vm()
md, wr = ir.read(slif)

In [ ]:
wr.rdr.setSeries(4)
wr.rdr.getImageCount()

In [ ]:
md

In [ ]:
rdr, xml_md = ir.img_reader(slif)

In [ ]:
rdr.setSeries(4)
rdr.getImageCount()

In [ ]:
xml_md.getPlaneCount(4), xml_md.getPlaneTheC(4, 110), xml_md.getPixelsSizeZ(4)

In [ ]:
rdr.getDimensionOrder()

In [ ]:
v = xml_md.getPixelsPhysicalSizeY(0)
v.value(), v.unit, ir.convert_value(v)

In [ ]:
md2, mdd2 = ir.read2(simg_tile)

In [ ]:
md2.keys(), md2["FolderCount"]

In [ ]:
md2["PlanePositionX"]

In [ ]:
extra_md, mdd = ir.get_md_dict(xml_md, simg_tile)

In [ ]:
extra_md["PlanePositionX"]

In [ ]:
ir.get_allvalues_grouped(xml_md, "getPixelsBigEndian", 1, True)

In [ ]:
md, mdd = ir.get_md_dict(xml_md)
len(md)

In [ ]:
md

In [ ]:
root = xml_md.getRoot()
im0 = root.getImage(0)
pixels = im0.getPixels()

In [ ]:
import io

plane = pixels.getPlane(3)

output = io.StringIO()
output.write(str(plane.getDeltaT()))
contents = output.getvalue()
output.close()

In [ ]:
output = io.StringIO()
output.write(str(plane.getDeltaT()))
contents = output.getvalue()
output.close()
contents

In [ ]:
md, missing_keys = ir.get_md_dict(xml_md)

In [ ]:
md["PixelsSizeZ"]

In [ ]:
getattr(xml_md, "getChannelExcitationWavelength")(0, 1)

## XML
getRoot is used in imgread-0.2.1

In [ ]:
wrapper, mdata = ir.img_reader(simg_tile)

In [ ]:
root = mdata.getRoot()

In [ ]:
instrument = root.getInstrument(0)
detector = instrument.getDetector(0)
detector.getModel()

In [ ]:
[m for m in root.methods if m[:3] == "get"]

In [ ]:
image = root.getImage(13)
[m for m in image.methods if m[:3] == "get"]

In [ ]:
obj = image.getObjectiveSettings()
[m for m in obj.methods if m[:3] == "get"]
obj.getID()

In [ ]:
ch = pixels.getChannel(0)
[m for m in ch.methods if m[:3] == "get"]

In [ ]:
lp = ch.getLightPath()
[m for m in lp.methods if m[:3] == "get"]

In [ ]:
pixels = image.getPixels()
[m for m in pixels.methods if m[:3] == "get"]

In [ ]:
pixels.getTimeIncrement()

In [ ]:
plane = pixels.getPlane(11)
[m for m in plane.methods if m[:3] == "get"]

In [ ]:
plane.getTheC(), plane.getDeltaT(), plane.getExposureTime()

In [ ]:
for k in root.methods:
    if k[:3] == "get":
        print(k)

In [ ]:
root.getInstrument(0).getDetector(0).getModel()

In [ ]:
mdata.getExperimentID(0)

In [ ]:
root.getExperiment(0)

In [ ]:
im = root.getImage(0)
obj = im.getObjectiveSettings()
obj.getID(), obj.getMedium(), obj.getRefractiveIndex(), obj.getCorrectionCollar(), obj.getObjective()

In [ ]:
mdata.getObjectiveID(0, 0) == obj.getID(), mdata.getObjectiveLensNA(0, 0)

## md vs root (to remember)

from md i can get info on camera, objectives ...

ut also key parameters like exposuretime, DeltaT, planePositionXYZ

In [ ]:
mdata.getObjectiveCount(0)

In [ ]:
metadata = mdata
# might be used to check if metadata are the same for all series
(
    metadata.getObjectiveCount(0),
    metadata.getObjectiveImmersion(0, 0),
    metadata.getPixelsDimensionOrder(0),
    metadata.getPixelsPhysicalSizeX(0),
    # metadata.getTiffDataIFD(0, 5), only fei
    metadata.getObjectiveLensNA(0, 0),
    metadata.getChannelCount(0),
    metadata.getDetectorSettingsBinning(0, 1),  # series, channel
    metadata.getDetectorModel(0, 0),
    metadata.getDetectorType(0, 0),
    metadata.getTiffDataCount(0),
    metadata.getPlaneExposureTime(0, 1),  # none.value() catch this error with try
    metadata.getPlaneDeltaT(0, 2).value(),
    metadata.getPlaneDeltaT(0, 11).value().doubleValue(),
    metadata.getPlanePositionX(0, 0),
    metadata.getScreenCount(),
    # metadata.getXMLAnnotationCount(),
)
# cerca tutti i ...Count()

In [ ]:
metadata.getXMLAnnotationValue(
    0
) == metadata.getRoot().getStructuredAnnotations().getXMLAnnotation(0).getValue()

In [ ]:
import bioformats

md = bioformats.get_omexml_metadata(slif)
from xml.etree import ElementTree as ETree

mdroot = ETree.fromstring(metadata.dumpXML())
mdroot = ETree.fromstring(
    metadata.getRoot().getStructuredAnnotations().getXMLAnnotation(0).getValue()
)
# mdroot = ETree.fromstring(md)

for a in mdroot:
    print((a.tag, a.attrib))
    for aa in a:
        print((" GGSSS ", aa.tag, aa.attrib))
        for aaa in aa:
            print((" GGSSS2 ", aaa.tag, aaa.attrib))

In [ ]:
import xmltodict

m1 = xmltodict.parse(metadata.dumpXML())
m2 = xmltodict.parse(bioformats.get_omexml_metadata(simg_tile))
m2["OME"]

In [ ]:
metalist = [
    (child.tag, [[child.attrib]])
    for child in ETree.fromstring(metadata.dumpXML()).iter()
]

In [ ]:
m = metalist[10]
m[0], m[1][0][0]

In [ ]:
for child in mdroot:
    if child.tag.endswith(""):
        for grandchild in child:
            print((grandchild.tag, grandchild.attrib))